# Задание по программированию: Линейная регрессия: прогноз оклада по описанию вакансии

## Вы научитесь:
- использовать линейную регрессию
- применять линейную регрессию к текстовым данным

## Введение
**Линейные методы** хорошо подходят для работы с **разреженными данными** — к таковым относятся, например, **тексты**. Это можно объяснить высокой **скоростью обучения** и небольшим количеством **параметров**, благодаря чему удается избежать **переобучения**.

Линейная регрессия имеет несколько разновидностей в зависимости от того, какой регуляризатор используется. Мы будем работать с гребневой регрессией, где применяется квадратичный, или L2-регуляризатор.

## Реализация в Scikit-Learn
Для извлечения **TF-IDF**-признаков из текстов воспользуйтесь классом `sklearn.feature_extraction.text.TfidfVectorizer`.

Для предсказания целевой переменной мы будем использовать гребневую регрессию, которая реализована в классе `sklearn.linear_model.Ridge`.

Обратите внимание, что признаки `LocationNormalized` и `ContractTime` являются строковыми, и поэтому с ними нельзя работать напрямую. Такие нечисловые признаки с неупорядоченными значениями называют **категориальными** или **номинальными**. Типичный подход к их обработке — кодирование категориального признака с **m** возможными значениями с помощью **m** бинарных признаков. Каждый бинарный признак соответствует одному из возможных значений категориального признака и является индикатором того, что на данном объекте он принимает данное значение. Данный подход иногда называют **one-hot-кодированием**. Воспользуйтесь им, чтобы перекодировать признаки `LocationNormalized` и `ContractTime`. Он уже реализован в классе `sklearn.feature_extraction.DictVectorizer`. Пример использования:

`from sklearn.feature_extraction import DictVectorizer
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))`

Вам понадобится производить замену пропущенных значений на специальные строковые величины (например, 'nan'). Для этого подходит следующий код:

`data_train['LocationNormalized'].fillna('nan', inplace=True)
data_train['ContractTime'].fillna('nan', inplace=True)`

In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from scipy import sparse

In [2]:
# функция для записи ответов
def write_answer(name, answer):
    with open('data/' + name + '.txt', 'w') as file:
        file.write(str(answer))

## Инструкция по выполнению
1) Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv (либо его заархивированную версию salary-train.zip).

In [3]:
train = pd.read_csv('data/salary-train.csv')
test = pd.read_csv('data/salary-test-mini.csv')

In [4]:
train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 4 columns):
FullDescription       60000 non-null object
LocationNormalized    60000 non-null object
ContractTime          44418 non-null object
SalaryNormalized      60000 non-null int64
dtypes: int64(1), object(3)
memory usage: 1.8+ MB


In [6]:
test

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
FullDescription       2 non-null object
LocationNormalized    2 non-null object
ContractTime          2 non-null object
SalaryNormalized      0 non-null float64
dtypes: float64(1), object(3)
memory usage: 144.0+ bytes


2) Проведите предобработку:
- Приведите тексты к нижнему регистру (text.lower()).
- Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. Для такой замены в строке text подходит следующий вызов: re.sub('[^a-zA-Z0-9]', ' ', text). Также можно воспользоваться методом replace у DataFrame, чтобы сразу преобразовать все тексты:

`train['FullDescription'] = train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)`

- Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).
- Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'. Код для этого был приведен выше.
- Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.
- Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.

**Приведите тексты к нижнему регистру (text.lower()).
Замените все, кроме букв и цифр, на пробелы:**

In [8]:
%%time
train['FullDescription'] = train['FullDescription'].apply(lambda s: s.lower()).replace('[^a-zA-Z0-9]', ' ', regex = True)
test['FullDescription'] = test['FullDescription'].apply(lambda s: s.lower()).replace('[^a-zA-Z0-9]', ' ', regex = True)

Wall time: 9.09 s


**Примените TfidfVectorizer:**

In [9]:
%%time
vectorizer = TfidfVectorizer(min_df=5)
X_train_text = vectorizer.fit_transform(train['FullDescription'])
X_test_text = vectorizer.transform(test['FullDescription'])

Wall time: 21 s


**Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan':**

In [10]:
train['LocationNormalized'].fillna('nan', inplace=True)
train['ContractTime'].fillna('nan', inplace=True)

test['LocationNormalized'].fillna('nan', inplace=True)
test['ContractTime'].fillna('nan', inplace=True)

**Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime**

In [11]:
enc = DictVectorizer()
X_train_categ = enc.fit_transform(train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(test[['LocationNormalized', 'ContractTime']].to_dict('records'))

**Объедините все полученные признаки в одну матрицу "объекты-признаки"**

In [12]:
X_train = sparse.hstack([X_train_text, X_train_categ])
X_test = sparse.hstack([X_test_text, X_test_categ])

y = train['SalaryNormalized']

3) Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. Целевая переменная записана в столбце SalaryNormalized.

In [14]:
ridge = Ridge(alpha=1, random_state=241)
ridge.fit(X_train, y)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

4) Постройте прогнозы для двух примеров из файла salary-test-mini.csv. Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.

In [15]:
predict = ridge.predict(X_test).round(2)
predict

array([ 56555.62,  37188.32])

In [16]:
write_answer('linreg_answer1', ' '.join(predict.astype(str)))